In [ ]:
from flow.core.params import VehicleParams, SumoCarFollowingParams, SumoLaneChangeParams
from flow.controllers import IDMController, ContinuousRouter, SimLaneChangeController
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams

# Example
from custom_network import myNetwork

# Ours
from custom_network import CustomBottleneckNetwork

In [ ]:
ADDITIONAL_NET_PARAMS = {
    "speed_limit": 20,
}

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)

In [ ]:
vehicles = VehicleParams()
vehicles.add(veh_id="human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=5)

sim_params = SumoParams(sim_step=0.1, 
                        render=True,
                        restart_instance=False)

initial_config = InitialConfig(bunching=0)

In [ ]:
# These are environment params, different from Net_params

from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS

env_params = EnvParams(
    additional_params=ADDITIONAL_ENV_PARAMS)

In [ ]:
from flow.core.experiment import Experiment

flow_params = dict(
    exp_tag='test_network',
    env_name=AccelEnv,
    network=CustomBottleneckNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 5000
exp = Experiment(flow_params)

# run 
_ = exp.run(1)

## Example

In [ ]:
ADDITIONAL_NET_PARAMS = {
    "radius": 40,
    "num_lanes": 1,
    "speed_limit": 30,
}

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)

In [ ]:
vehicles = VehicleParams()
vehicles.add(veh_id="human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=22)

sim_params = SumoParams(sim_step=0.1, render=True)

initial_config = InitialConfig(bunching=40)

In [ ]:
from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

In [ ]:
from flow.core.experiment import Experiment

flow_params = dict(
    exp_tag='test_network',
    env_name=AccelEnv,
    network=myNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 1500
exp = Experiment(flow_params)

# run 
_ = exp.run(1)

## Inspired from Cathy's Intersection

In [1]:
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams, \
    InFlows, SumoCarFollowingParams, SumoLaneChangeParams
from flow.core.params import TrafficLightParams
from flow.core.params import VehicleParams
from flow.controllers import RLController, ContinuousRouter, \
    SimLaneChangeController
from flow.envs import BottleneckDesiredVelocityEnv
from flow.networks import BottleneckNetwork

In [2]:
# time horizon of a single rollout
HORIZON = 5000
# number of parallel workers
N_CPUS = 1
# number of rollouts per training iteration
N_ROLLOUTS = N_CPUS * 4

SCALING = 1
NUM_LANES = 4 * SCALING  # number of lanes in the widest highway
DISABLE_TB = True
DISABLE_RAMP_METER = True
AV_FRAC = 0.10

vehicles = VehicleParams()

In [3]:
vehicles.add(
    veh_id="human",
    lane_change_controller=(SimLaneChangeController, {}),
    routing_controller=(ContinuousRouter, {}),
    car_following_params=SumoCarFollowingParams(
        speed_mode="all_checks",
    ),
    lane_change_params=SumoLaneChangeParams(
        lane_change_mode=0,
    ),
    num_vehicles=1 * SCALING)
vehicles.add(
    veh_id="followerstopper",
    acceleration_controller=(RLController, {}),
    lane_change_controller=(SimLaneChangeController, {}),
    routing_controller=(ContinuousRouter, {}),
    car_following_params=SumoCarFollowingParams(
        speed_mode=9,
    ),
    lane_change_params=SumoLaneChangeParams(
        lane_change_mode=0,
    ),
    num_vehicles=1 * SCALING)

In [4]:
controlled_segments = [("1", 1, False), ("2", 2, True), ("3", 2, True),
                       ("4", 2, True), ("5", 1, False)]
num_observed_segments = [("1", 1), ("2", 3), ("3", 3), ("4", 3), ("5", 1)]
additional_env_params = {
    "target_velocity": 40,
    "disable_tb": True,
    "disable_ramp_metering": True,
    "controlled_segments": controlled_segments,
    "symmetric": False,
    "observed_segments": num_observed_segments,
    "reset_inflow": False,
    "lane_change_duration": 5,
    "max_accel": 3,
    "max_decel": 3,
    "inflow_range": [1000, 2000]
}

In [5]:
flow_rate = 10000

In [6]:
# percentage of flow coming out of each lane
inflow = InFlows()
inflow.add(
    veh_type="human",
    edge="1",
    vehs_per_hour=flow_rate * (1 - AV_FRAC),
    departLane="random",
    departSpeed=10)
inflow.add(
    veh_type="followerstopper",
    edge="1",
    vehs_per_hour=flow_rate * AV_FRAC,
    departLane="random",
    departSpeed=10)

traffic_lights = TrafficLightParams()
if not DISABLE_TB:
    traffic_lights.add(node_id="2")
if not DISABLE_RAMP_METER:
    traffic_lights.add(node_id="3")

additional_net_params = {"scaling": SCALING, "speed_limit": 23}
net_params = NetParams(
    inflows=inflow,
    additional_params=additional_net_params)

/home/dh127-pc3/Desktop/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departLane in InFlows is deprecated, use depart_lane instead.
  PendingDeprecationWarning
/home/dh127-pc3/Desktop/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departSpeed in InFlows is deprecated, use depart_speed instead.
  PendingDeprecationWarning
/home/dh127-pc3/Desktop/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departLane in InFlows is deprecated, use depart_lane instead.
  PendingDeprecationWarning
/home/dh127-pc3/Desktop/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departSpeed in InFlows is deprecated, use depart_speed instead.
  PendingDeprecationWarning


In [7]:

flow_params = dict(
    # name of the experiment
    exp_tag="DesiredVelocity",

    # name of the flow environment the experiment is running on
    env_name=BottleneckDesiredVelocityEnv,

    # name of the network class the experiment is running on
    network=BottleneckNetwork,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.5,
        render=True,
        print_warnings=False,
        restart_instance=True,
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        warmup_steps=40,
        sims_per_step=1,
        horizon=HORIZON,
        additional_params=additional_env_params,
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=NetParams(
        inflows=inflow,
        additional_params=additional_net_params,
    ),

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=InitialConfig(
        spacing="uniform",
        min_gap=5,
        lanes_distribution=float("inf"),
        edges_distribution=["2", "3", "4", "5"],
    ),

    # traffic lights to be introduced to specific nodes (see
    # flow.core.params.TrafficLightParams)
    tls=traffic_lights,
)

In [8]:
from flow.core.experiment import Experiment

exp = Experiment(flow_params)

# run 
_ = exp.run(1)

FatalTraCIError: connection closed by SUMO